# Attention Is All You Need

## About this Notebook
This notebook focuses on training a Transformer model to translate sentences from French to English. The Transformer model, proposed in the paper "Attention is All You Need" by Vaswani et al.

We refer to the article [The Annotated Transformer](http://nlp.seas.harvard.edu/annotated-transformer/#embeddings-and-softmax) for the implementation details of the Transformer model.

## Dataset
We use the [English-French Translation Dataset](https://www.kaggle.com/datasets/dhruvildave/en-fr-translation-dataset/) available on Kaggle for training this model. This dataset contains more than 22.5 million sentence pairs for translation. However, due to computational constraints and to avoid timeout during training on Kaggle, we only train on 1% of the dataset.

## Training
The training is performed using **2 x T4 GPUs** with PyTorch's `DataParallel` for distributed computing.


In [68]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [69]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
import math 
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device :", device)

device : cuda


# Text Preprocessing 🤖

NLP involves a lot of data wrangling. To stick to the simple but insightful example I won't use the usual convenience functions (for example by hugginface) to create the text-pipeline. 
Tokenizer, Vocabulary, Batch-Makers are coded with good ol' python. 

*The code for the tokenizer and vocab builder are from [this](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) excellent torch tutorial!*

In [70]:
SOS_token = 0
EOS_token = 1
PAD_token = 2
MAX_LENGTH = 50

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"PAD"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

# def readLangs(lang1, lang2, reverse=False):
#     lines = open('../input/english-french/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
#         read().strip().split('\n')

#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

#     # Reverse pairs, make Lang instances
#     if reverse:
#         pairs = [list(reversed(p)) for p in pairs]
#         input_lang = Lang(lang2)
#         output_lang = Lang(lang1)
#     else:
#         input_lang = Lang(lang1)
#         output_lang = Lang(lang2)

#     return input_lang, output_lang, pairs

def readLangs(lang1, lang2, reverse=False):
    df = pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')

    # Sample 1% of your dataframe
    df_sample = df.sample(frac=0.01)

    # Split every line into pairs and normalize
    pairs = [[normalizeString(str(s)) for s in l] for l in df_sample.values]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [71]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Read 225204 sentence pairs
Trimmed to 192480 sentence pairs
Counting words...
Counted words:
fra 99082
eng 83591
['version facile a imprimer chaire de recherche du canada en geometrie differentielle et topologie universite de montreal niveau mars date de renouvellement mars sciences naturelles et genie poste lalonde dms .umontreal .ca', 'print friendly version canada research chair in differential geometry and topology universite de montreal tier march renewed march natural sciences and engineering poste lalonde dms .umontreal .ca']


### From sentence to indexes and batches

Bellow are the functions that transform sentences into lists of indexes and then creates batches. 

In [72]:
def indexes_from_sentence(lang, sentence):
    idxs = [lang.word2index[word] for word in sentence.split(' ')]
    idxs.append(EOS_token)
    idxs.insert(SOS_token,0)
    return idxs
    
def batch_from_pairs(pairs):
    batch_inp = [indexes_from_sentence(input_lang, p[0]) for p in pairs]
    longest_seq = max([len(seq) for seq in batch_inp])
    batch_inp = [seq+[PAD_token]*(longest_seq-len(seq)) for seq in batch_inp]
    input_tensor = torch.tensor(batch_inp, dtype=torch.long, device=device)
    
    batch_trg = [indexes_from_sentence(output_lang, p[1]) for p in pairs]
    longest_seq = max([len(seq) for seq in batch_trg])
    batch_trg = [seq+[PAD_token]*(longest_seq-len(seq)) for seq in batch_trg]
    target_tensor = torch.tensor(batch_trg, dtype=torch.long, device=device)    
    
    return input_tensor,target_tensor

In [73]:
input_tensor,target_tensor = batch_from_pairs(pairs)
print("input lang:")
print(input_lang)
print("---"*40)
print("input tensor:")
print(input_tensor)
print("---"*40)
print("input tensor shape:")
print(input_tensor.shape)
print()
print("output lang:")
print(output_lang)
print("---"*40)
print("target tensor:")
print(target_tensor)
print("---"*40)
print("target tensor shape:")
print(target_tensor.shape)

input lang:
------------------------------------------------------------------------------------------------------------------------
input tensor:
tensor([[    0,     3,     4,  ...,     2,     2,     2],
        [    0,    29,    30,  ...,     2,     2,     2],
        [    0,    51,     4,  ...,     2,     2,     2],
        ...,
        [    0, 29455,   153,  ...,     2,     2,     2],
        [    0,    59,   421,  ...,     2,     2,     2],
        [    0,    44,   704,  ...,     2,     2,     2]], device='cuda:0')
------------------------------------------------------------------------------------------------------------------------
input tensor shape:
torch.Size([192480, 51])

output lang:
------------------------------------------------------------------------------------------------------------------------
target tensor:
tensor([[    0,     3,     4,  ...,     2,     2,     2],
        [    0,    26,    27,  ...,     2,     2,     2],
        [    0,    43,    26,  ...,     2,

# Implementing the Transformer 

![Model Architecture](https://www.researchgate.net/profile/Dennis-Gannon-2/publication/339390384/figure/fig1/AS:860759328321536@1582232424168/The-transformer-model-from-Attention-is-all-you-need-Viswani-et-al.jpg)

### Some further notes to the illustration

* It is encoder-decoder architecture. 
* The encoder creates features of the input sequence.
* The decoder uses the output of the encoder to decode the target sequence.
* The decoder uses the shifted targets during training. During prediction the outputs are fed back as the new inputs.
* Both encoder are using stacked self-attention and point-wise, fully connected layers.

Let's go through it step by step.

###  Scaled dot-product attention
The authors wrote:

> We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of
queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the
query with all keys, divide each by √
dk, and apply a softmax function to obtain the weights on the
values.
In practice, we compute the attention function on a set of queries simultaneously, packed together
into a matrix Q. The keys and values are also packed together into matrices K and V . We compute
the matrix of outputs as:

![Attention function](https://miro.medium.com/max/720/1*P9sV1xXM10t943bXy_G9yg.png)

### Multihead attention

> Instead of performing a single attention function with dmodel-dimensional keys, values and queries,
we found it beneficial to linearly project the queries, keys and values h times with different, learned
linear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of
queries, keys and values we then perform the attention function in parallel, yielding dv-dimensional
output values. These are concatenated and once again projected, resulting in the final values ... Multi-head attention allows the model to jointly attend to information from different representation
subspaces at different positions. With a single attention head, averaging inhibits this.


In [74]:
import torch
import torch.nn as nn

############################################
# STUDENT TASK: Implement Scaled Dot-Product Attention
############################################
def attention(q, k, v, dropout, mask=None):
    """
    Implement the scaled dot-product attention mechanism.

    Steps to implement:
    1. Compute raw attention scores using matrix multiplication: q @ kᵀ
    2. Scale the scores by dividing by sqrt(d_k)
    3. If a mask is provided, set masked positions to a very negative value
       so they become zero after softmax.
    4. Apply softmax over the last dimension to obtain attention weights.
    5. Apply dropout to the attention weights.
    6. Multiply the attention weights by v to obtain the output.

    Return:
        The weighted sum of values.
    """
    # scores = q @ k.transpose(-1, -2) / (q.size(-1) ** 0.5)
    # if mask is not None:
    #     scores = scores.masked_fill(mask == 0, float('-inf'))
    # attn_weights = F.softmax(scores, dim=-1)
    # if dropout is not None:
    #     attn_weights = dropout(attn_weights)
    # return attn_weights @ v
    dk = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2, -1)) / (dk**0.5)
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    attn_weights = torch.softmax(scores, dim=-1)
    attn_weights = dropout(attn_weights)
    return torch.matmul(attn_weights, v)


############################################
# STUDENT TASK: Implement Multi-Head Attention
############################################
class MultiHeaderAttention(nn.Module):
    """
    Multi-Head Attention layer used in Transformer models.

    Students must:
    1. Complete all linear projections for q, k, v.
    2. Reshape the projections into multiple heads.
    3. Apply the attention() function across all heads.
    4. Concatenate the heads and apply the final output projection.
    5. Properly handle masking across heads.
    """
    
    def __init__(self, d_model, dropout, n_heads=8, dk=64, dv=64):
        super(MultiHeaderAttention, self).__init__()
        # STUDENT TASK:
        # Define linear layers to project inputs into q, k, v vectors
        # Each should project to (n_heads * dk) or (n_heads * dv)
        # Also define dropout and the final output linear layer.
        self.d_model    = d_model
        self.n_heads    = n_heads
        self.dk         = dk
        self.dv         = dv
        self.q_linear   = nn.Linear(d_model, n_heads * dk)
        self.k_linear   = nn.Linear(d_model, n_heads * dk)
        self.v_linear   = nn.Linear(d_model, n_heads * dv)
        self.attn       = attention
        self.dropout    = nn.Dropout(dropout)
        self.out_linear = nn.Linear(n_heads * dv, d_model)

    def forward(self, k, v, q, mask=None):
        """
        Forward pass of Multi-Head Attention.

        Steps to implement:
        1. Get batch size and sequence lengths.
        2. Apply the q, k, v linear projections.
        3. Reshape each projected tensor to (batch, seq_len, heads, dk or dv).
        4. Move the head dimension before sequence length.
        5. Apply the attention() function over heads.
        6. Re-combine heads by transposing back and reshaping.
        7. Apply the final output projection.
        8. Apply mask correctly by expanding across heads.
        """
        batch_size = q.size(0)
        seq_len_q = q.size(1)
        seq_len_k = k.size(1)
        seq_len_v = v.size(1)
        query = self.q_linear(q).view(batch_size, seq_len_q, self.n_heads, self.dk).transpose(1,2)
        key = self.k_linear(k).view(batch_size, seq_len_k, self.n_heads, self.dk).transpose(1,2)
        value = self.v_linear(v).view(batch_size, seq_len_v, self.n_heads, self.dv).transpose(1,2)
        out = self.attn(query, key, value, self.dropout, mask)
        out = out.transpose(1,2).contiguous().view(batch_size, seq_len_q, self.n_heads * self.dv)
        out = self.out_linear(out)
        return out

### Positional encoding

The model needs to know the order of the words in a sentence. (Or more precisely, the order of the embeddings)
In the paper they do this by adding a cyclical signal to the word embeddings.

The authors write:
> Since our model contains no recurrence and no convolution, in order for the model to make use of the
> order of the sequence, we must inject some information about the relative or absolute position of the"

and

> To this end, we add "positional encodings" to the input embeddings at the
bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel
as the embeddings, so that the two can be summed. There are many choices of positional encodings,
learned and fixed [9].
In this work, we use sine and cosine functions of different frequencies:
PE(pos,2i) = sin(pos/10000* 2i/dmodel) PE(pos,2i+1) = cos(pos/10000 * 2i/dmodel)
where pos is the position and i is the dimension. That is, each dimension of the positional encoding
corresponds to a sinusoid. The wavelengths form a geometric progression from 2π to 10000 · 2π. 

In [75]:
import torch
import torch.nn as nn
import math

############################################
# STUDENT TASK: Implement Positional Encoding
############################################
class PositionEncoding(nn.Module):
    """
    Absolute positional encoding used in Transformer models.
    
    Students must complete:
    - pos_encoding(): compute sinusoidal positional embedding for a given position.
    - tensor_pos_encoding(): generate a table of embeddings for all positions.
    - forward(): add positional embeddings to the input.
    """

    def __init__(self, max_len, d_model):
        super(PositionEncoding, self).__init__()

        # STUDENT TASK:
        # 1. Store max_len (optionally add padding room).
        # 2. Register a buffer named "pos_table" containing the
        #    precomputed positional encodings for all positions.
        #
        # Hint: use self.register_buffer(name, tensor)
        self.max_len = max_len
        self.d_model = d_model
        pos_table = self.tensor_pos_encoding(max_len, d_model)
        self.register_buffer('pos_table', pos_table)

    @staticmethod
    def pos_encoding(pos, k):
        """
        STUDENT TASK:

        Compute the sinusoidal positional encoding for a single position `pos`
        and dimension `k`.

        For dimension index i (0 ≤ i < k):
            If i is even:   use sin( pos / (10000^(i/k)) )
            If i is odd:    use cos( pos / (10000^(i/k)) )

        Return:
            A list (or tensor) of length k containing the positional embedding.
        """
        div_term = torch.exp(torch.arange(0, k, 2, dtype=torch.float, device=device) * -(math.log(10000.0) / k))
        pe = torch.zeros(k, device=device)
        # pe[0::2] = torch.sin(pos * div_term[:len(pe[0::2])])
        # pe[1::2] = torch.cos(pos * div_term[:len(pe[1::2])])
        pe[0::2] = torch.sin(pos * div_term)
        pe[1::2] = torch.cos(pos * div_term)
        return pe

    def tensor_pos_encoding(self, max_len, dim):
        """
        STUDENT TASK:

        Build the full positional encoding table:
        - Shape: (max_len, dim)
        - Each row i contains the positional encoding for position i.

        Return:
            A tensor containing all positional encodings.
        """
        pe_table = torch.zeros(max_len, dim, device=device)
        for pos in range(max_len):
            pe_table[pos] = self.pos_encoding(pos, dim)
        return pe_table

    def forward(self, x):
        """
        STUDENT TASK:

        Add positional embeddings to the input tensor.

        Steps:
        1. Extract the sequence length from x.
        2. Select the corresponding rows from pos_table.
        3. Expand positional encodings to match batch size.
        4. Add them to x and return the result.

        Note:
        - Remember to detach() the table to avoid gradient computation.
        """
        batch_size, seq_len, d_model = x.size()
        pos_enc = self.pos_table[:seq_len, :]
        pos_enc = pos_enc.unsqueeze(0).expand(batch_size, -1, -1)
        return x + pos_enc.detach()

### Subylayer connection

Sometimes it's good to learn from the past. That's basically what residual connections do. 

> We employ a residual connection [11] around each of
the two sub-layers, followed by layer normalization [1]. That is, the output of each sub-layer is
LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer
itself. 

Concering dropout:
> We apply dropout [33] to the output of each sub-layer, before it is added to the
sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the
positional encodings in both the encoder and decoder stacks. For the base model, we use a rate of
Pdrop = 0.1.

In [76]:
import torch
import torch.nn as nn

############################################
# STUDENT TASK: Implement SublayerConnection
############################################
class SublayerConnection(nn.Module):
    """
    Implements the 'Add & Norm' operation used throughout the Transformer.

    Students must implement:
    - A LayerNorm operation
    - A Dropout layer
    - A forward() method that:
        1. Applies LayerNorm to the input
        2. Passes the normalized input through a given sublayer (function)
        3. Applies dropout
        4. Adds the original input (residual connection)
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()

        # STUDENT TASK:
        # 1. Create a LayerNorm of dimension `size`
        # 2. Create a Dropout layer with probability `dropout`
        self.layer_norm = nn.LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        """
        STUDENT TASK:

        Implement the forward pass:

        Steps:
        1. Normalize input x using the layer norm.
        2. Pass normalized x to the provided `sublayer` function.
           (The sublayer will typically be attention or a feed-forward network.)
        3. Apply dropout to the sublayer’s output.
        4. Add the result back to the original input (residual connection).

        Return:
            The output of x + dropout(sublayer(norm(x)))
        """
        return x + self.dropout(sublayer(self.layer_norm(x)))

### Encoder

The encoder will take our French sentences an *encode* it. Imagine that you are engineering features for your favorite gradient booster model. The output of the encoder is basically the features of our French sentences, ready to be fed into the decoder.

The authors description of the encoder:
> The encoder is composed of a stack of N = 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, positionwise fully connected feed-forward network. We employ a residual connection [11] around each of
the two sub-layers, followed by layer normalization [1]. That is, the output of each sub-layer is
LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512.

The application of attention in the encoder is described as followed:
> The encoder contains self-attention layers. In a self-attention layer all of the keys, values
and queries come from the same place, in this case, the output of the previous layer in the
encoder. Each position in the encoder can attend to all positions in the previous layer of the
encoder.

In [77]:
############################################
# STUDENT TASK: Implement a single Encoder Layer
############################################
class EncoderLayer(nn.Module):
    """
    One layer of the Transformer Encoder.

    This layer contains:
    - Multi-head self-attention sublayer
    - Feed-forward neural network sublayer
    - Two SublayerConnection modules
    """

    def __init__(self, d_model, n_hidden, dropout):
        super(EncoderLayer, self).__init__()

        # STUDENT TASK:
        # 1. Create a MultiHeaderAttention module for self-attention
        # 2. Create a feed-forward network:
        #       Linear(d_model → n_hidden)
        #       ReLU
        #       Dropout
        #       Linear(n_hidden → d_model)
        # 3. Create two SublayerConnection modules:
        #       - One for self-attention
        #       - One for feed-forward
        self.d_model = d_model
        self.self_attn = MultiHeaderAttention(d_model, dropout)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, n_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(n_hidden, d_model)
        )
        self.sublayers = nn.ModuleList([SublayerConnection(d_model, dropout) for _ in range(2)])

    def forward(self, x, mask):
        """
        STUDENT TASK:

        Forward pass consists of:
        1. Applying self-attention inside a SublayerConnection:
            sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))

        2. Applying feed-forward inside the second SublayerConnection:
            sublayer[1](output_of_step_1, self.feed_forward)

        Return:
            The transformed output tensor.
        """
        out = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, mask))
        out = self.sublayers[1](out, self.ffn)
        return out

In [78]:
import torch
import torch.nn as nn

############################################
# STUDENT TASK: Implement the Transformer Encoder
############################################
class Encoder(nn.Module):
    """
    Full Transformer Encoder consisting of:
    - Token embeddings
    - Positional encodings
    - N stacked EncoderLayers

    Students must implement:
    1. The embedding layer for input tokens
    2. The positional encoding addition
    3. The loop over N encoder layers
    4. A final LayerNorm
    """

    def __init__(self, n_input_vocab, d_model, n_hidden, n_layers, dropout):
        super().__init__()

        # STUDENT TASK:
        # 1. Create an embedding layer of size (n_input_vocab, d_model)
        #    - Remember to specify padding_idx
        # 2. Add a dropout layer
        # 3. Add a final LayerNorm for output normalization
        # 4. Create a list of n_layers EncoderLayer modules
        # 5. Instantiate a PositionalEncoding module
        self.d_model = d_model
        self.embedding = nn.Embedding(n_input_vocab, d_model, padding_idx=PAD_token)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model)
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_hidden, dropout) for _ in range(n_layers)])
        self.pos_encoding = PositionEncoding(MAX_LENGTH+2, d_model)

    def forward(self, x, mask):
        """
        STUDENT TASK:

        Steps:
        1. Convert token indices into embeddings.
        2. Scale embeddings by sqrt(d_model).
        3. Add positional encodings.
        4. Pass through each EncoderLayer sequentially.
        5. Apply final normalization.

        Arguments:
            x: (batch_size, sequence_length)
            mask: (batch_size, sequence_length) or broadcastable

        Return:
            Encoded representation of shape (batch_size, sequence_length, d_model)
        """
        out = self.embedding(x) / (self.d_model**0.5)
        out = self.pos_encoding(out)
        out = self.dropout(out)
        for layer in self.layers:
            out = layer(out, mask)
        out = self.layer_norm(out)
        print("Encoder output shape:", out.shape)
        return out

### Decoder

The decoder will take the encoder output (features of the French sentences) and the target sequence (here the English sentences) and decode it. The decoder output is then fed into a linear layer whose job it is to predict the next word of each vector of the sequence. (See how the labels are made during training to understand why.) 

> The decoder is also composed of a stack of N = 6 identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions less than i.


The first application of the attention in the decoder (sublayer1) is described as follows:
> Similarly, self-attention layers in the decoder allow each position in the decoder to attend to
all positions in the decoder up to and including that position. We need to prevent leftward
information flow in the decoder to preserve the auto-regressive property. We implement this
inside of scaled dot-product attention by masking out (setting to −∞) all values in the input
of the softmax which correspond to illegal connections. See Figure 2.

The second application of the attention in the decoder (sublayer2) is described as follows:
> In "encoder-decoder attention" layers, the queries come from the previous decoder layer,
and the memory keys and values come from the output of the encoder. This allows every
position in the decoder to attend over all positions in the input sequence. 

In [79]:
############################################
# STUDENT TASK: Implement a single Decoder Layer
############################################
class DecoderLayer(nn.Module):
    """
    One layer of the Transformer Decoder.

    Contains three sublayers:
    1. Masked self-attention (target → target)
    2. Encoder-decoder attention (target → source)
    3. Feed-forward network

    Each sublayer is wrapped in SublayerConnection.
    """

    def __init__(self, d_model, n_hidden, dropout):
        super(DecoderLayer, self).__init__()

        # STUDENT TASK:
        # 1. Create MultiHeaderAttention for:
        #       - self-attention (target attends to target)
        #       - encoder-decoder attention (target attends to memory)
        # 2. Create a feed-forward network:
        #       Linear(d_model → n_hidden)
        #       ReLU
        #       Dropout
        #       Linear(n_hidden → d_model)
        # 3. Create THREE SublayerConnection modules
        self.self_attn = MultiHeaderAttention(d_model, dropout)
        self.src_attn_attn = MultiHeaderAttention(d_model, dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, n_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(n_hidden, d_model)
        )
        self.sublayers = nn.ModuleList([SublayerConnection(d_model, dropout) for _ in range(3)])

    def forward(self, x, memory, tgt_mask, src_mask):
        """
        STUDENT TASK:

        Decoder forward pass:

        Given:
            x: (batch, tgt_len, d_model)
            memory: encoder outputs (batch, src_len, d_model)
            tgt_mask: future mask (for self-attention)
            src_mask: padding mask (for encoder-decoder attention)

        Steps:
        1. Masked self-attention:
               sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))

        2. Encoder-decoder attention:
               sublayer[1](output, lambda x: self.src_attn(memory, memory, x, src_mask))

        3. Feed-forward network:
               sublayer[2](output, self.feed_forward)

        Return:
            Final transformed representation.
        """
        out = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        out = self.sublayers[1](out, lambda x: self.src_attn_attn(memory, memory, x, src_mask))
        out = self.sublayers[2](out, self.feed_forward)
        return out

In [80]:
import torch
import torch.nn as nn

############################################
# STUDENT TASK: Implement the Transformer Decoder
############################################
class Decoder(nn.Module):
    """
    Transformer Decoder:
    - Token embedding
    - Positional encoding
    - N stacked DecoderLayers
    - Final LayerNorm

    Students must:
    1. Create an embedding layer for target vocabulary.
    2. Add positional encodings.
    3. Stack multiple DecoderLayer modules.
    4. Apply LayerNorm to the final output.
    """

    def __init__(self, n_target_vocab, d_model, n_hidden, n_layers, dropout):
        super().__init__()

        # STUDENT TASK:
        # 1. Create target token embedding (with padding index).
        # 2. Create dropout and final layer norm.
        # 3. Create a ModuleList of n_layers DecoderLayer objects.
        # 4. Create a PositionalEncoding module.
        self.d_model = d_model
        self.embedding = nn.Embedding(n_target_vocab, d_model, padding_idx=PAD_token)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model)
        self.layers = nn.ModuleList([DecoderLayer(d_model, n_hidden, dropout) for _ in range(n_layers)])
        self.pos_encoding = PositionEncoding(MAX_LENGTH+2, d_model)

    def forward(self, x, encoder_outputs, self_attn_mask, enc_dec_mask):
        """
        STUDENT TASK:

        Steps:
        1. Convert target tokens to embeddings.
        2. Scale embeddings by sqrt(d_model).
        3. Add positional encodings.
        4. Pass the sequence through each decoder layer:
            - Each layer performs masked self-attention
            - Then encoder-decoder attention
            - Then a feed-forward network
        5. Apply final LayerNorm.

        Arguments:
            x: target token indices (batch, tgt_len)
            encoder_outputs: (batch, src_len, d_model)
            self_attn_mask: target-side future mask (causal)
            enc_dec_mask: encoder-to-decoder attention mask
        """
        out = self.embedding(x) / (self.d_model**0.5)
        out = self.pos_encoding(out)
        out = self.dropout(out)
        for layer in self.layers:
            out = layer(out, encoder_outputs, self_attn_mask, enc_dec_mask)
        out = self.layer_norm(out)
        return out

## The Transfomer 

Now we have all the pieces for the transformer. Here, the encoder and decoder are instanciated, the masks that prevent looking at irrelevant (pad tokens) or illegal (looking ahead) content are generated. A linear layer outputs the final guesses of the word distributions for the whole sequence. Note that the Softmax functions are not implemented. This is because the Crossentropy Loss expects the raw output. 

In [81]:
import torch
import torch.nn as nn

############################################
# STUDENT TASK: Implement the Transformer
############################################
class Transformer(nn.Module):
    """
    Full Transformer model composed of:
    - Encoder
    - Decoder
    - Final linear projection to vocabulary logits

    Students must implement:
    1. Initialization of encoder, decoder, and final output layer.
    2. Padding mask for both encoder and decoder.
    3. Causal target mask (for autoregressive decoding).
    4. Forward pass combining all components.
    """

    def __init__(self, d_model, n_input_vocab, n_target_vocab, n_hidden, n_layers, dropout):
        super().__init__()

        # STUDENT TASK:
        # 1. Instantiate an Encoder with the given arguments.
        # 2. Instantiate a Decoder.
        # 3. Add a final Linear layer projecting decoder outputs to target vocabulary size.
        self.encoder = Encoder(n_input_vocab, d_model, n_hidden, n_layers, dropout)
        self.decoder = Decoder(n_target_vocab, d_model, n_hidden, n_layers, dropout)
        self.final_linear = nn.Linear(d_model, n_target_vocab)

    ############################################
    # STUDENT TASK: Padding Mask
    ############################################
    def get_pad_mask(self, seq):
        """
        Build a padding mask for sequences.

        Students must:
        - Return a mask of shape (batch, 1, seq_len)
        - Mask = True where token != PAD_token
        - Mask = False where token == PAD_token
        """
        return (seq != PAD_token).unsqueeze(1).to(device)

    ############################################
    # STUDENT TASK: Target Mask (Causal Mask)
    ############################################
    def get_target_mask(self, target_seq):
        """
        Build a causal mask to prevent attention to future tokens.

        Students must:
        - Use torch.tril() to create a lower-triangular matrix
        - Shape: (1, tgt_len, tgt_len)
        - Return a boolean mask suitable for attention
        """
        tgt_len = target_seq.size(1)
        mask = torch.tril(torch.ones(1, tgt_len, tgt_len, device=device)).bool()
        return mask

    ############################################
    # STUDENT TASK: Forward Pass
    ############################################
    def forward(self, input_seq, target_seq):
        """
        Complete the full forward pass of the Transformer.

        Steps:
        1. Compute source padding mask.
        2. Compute target padding mask.
        3. Compute causal mask and combine with padding mask.
        4. Pass input_seq through the encoder.
        5. Pass target_seq and encoder output into the decoder.
        6. Apply the final linear projection to get vocab logits.

        Returns:
            Output logits of shape (batch, tgt_len, n_target_vocab)
        """
        src_pad_mask = self.get_pad_mask(input_seq)
        tgt_pad_mask = self.get_pad_mask(target_seq)
        tgt_casual_mask = self.get_target_mask(target_seq)
        tgt_mask = tgt_pad_mask & tgt_casual_mask
        encoder_outputs = self.encoder(input_seq, src_pad_mask)
        decoder_outputs = self.decoder(target_seq, encoder_outputs, tgt_mask, src_pad_mask)
        output_logits = self.final_linear(decoder_outputs)
        return output_logits

# Training & Random Tests

It's training time. First we shuffle the dataset and split it into training and test partitions. Then we instatiate the model and set up the optimizer and scheduler. Finally we run the training loop. We get a hot cup of coffee, lean back and look mesmerized at the ever decreasing loss. ☕️

In [82]:
#splitting the data into train and test datasets

np.random.shuffle(pairs)
train = pairs[:-1000]
test = pairs[-1000:]

#the model with the parameter-values of the paper
transformer1 = Transformer(d_model=512, 
                            n_input_vocab=input_lang.n_words, 
                            n_target_vocab= output_lang.n_words,
                            n_hidden = 2048,
                            n_layers=6, 
                            dropout=0.1).to(device)

transformer1 = nn.DataParallel(transformer1)

for p in transformer1.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

### Optimizer 

> We used the Adam optimizer [20] with β1 = 0.9, β2 = 0.98 and  = 10−9
. We varied the learning
rate over the course of training, according to the formula:<br>
`lrate = d_model**−0.5 model · min(step_num−0.5, step_num · warmup_steps−1.5)` (3)<br>
This corresponds to increasing the learning rate linearly for the first warmup_steps training steps,
and decreasing it thereafter proportionally to the inverse square root of the step number. We used
warmup_steps = 4000.

In [83]:
#the optimizer 
lr= 1
opt1 = optim.Adam(transformer1.parameters(),lr=lr, betas=(0.9, 0.98), eps=1e-09)

def lr_rate(step_num, d_model, factor, warmup_steps):
    step_num =max(1,step_num)
    return factor * (
        d_model ** (-0.5) * min(step_num ** (-0.5), step_num * warmup_steps ** (-1.5))
    )

lr_scheduler = LambdaLR(
    optimizer=opt1,
    lr_lambda=lambda step_num: lr_rate(
        step_num, 512, factor=1, warmup_steps=4000
    ),
)

### Training and evaluation functions

Let's define a function to test the model with random samples from the test set. This will make the learning process more intuitive. Ultimately, we want to see the model translate something it has not seen before and the better the quality of this.

In [84]:
def pred(input_seq, model):
    outputs = [SOS_token]
    
    loss = 0
    for i in range(MAX_LENGTH):
        target_seq = torch.tensor([outputs],device=device)
        output = model(input_seq,target_seq)
        probs = F.softmax(output,dim=2)
        word_pred = torch.argmax(probs[:,-1,:],dim=1)

        outputs.append(word_pred.item())

        if word_pred.item()== EOS_token:
            break
    
    return outputs[1:]

def random_model_testing(n_examples,model):
    batch_sz=1
    test_samples = [random.choice(test) for i in range(n_samples)]
    print("Random Tests")
    print("*"*30)
    for i in range(0,len(test_samples[:n_examples]),batch_sz):
        input_tensor, output_tensor = batch_from_pairs(test_samples[i:i+batch_sz])
        out = pred(input_tensor, model)
        print("Pred: ", " ".join([output_lang.index2word[i] for i in out]), "True: ",test_samples[i][1])
    print("*"*30)

In [85]:
def train_batch(input_seq, target_seq, model, optimizer,scheduler):
    target, truth = target_seq[:,:-1], target_seq[:,1:]
    pred = model(input_seq,target)
    
    loss = F.cross_entropy(pred.view(-1,output_lang.n_words), truth.reshape(-1),reduction='sum',label_smoothing=0.1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()

    return loss.item()

### Let's train our transfomer!

Bellow is a simple training loop. Each epoch we feed the input and target batches to the model, calculate the loss and backpropagate.

In [ ]:
n_samples=21000
epochs =200
batch_sz=128

for e in range(epochs):
    loss = 0
    train_samples = [random.choice(train) for i in range(n_samples)]
    for i in range(0,len(train_samples),batch_sz):
        input_tensor, output_tensor = batch_from_pairs(train_samples[i:i+batch_sz])
        loss += train_batch(input_tensor, output_tensor, transformer1, opt1,lr_scheduler)
    print(f"Epoch {e}/{epochs} | loss: {round(loss/n_samples,2)} | learning rate: {round(lr_scheduler.get_last_lr()[0],6)}")
    #random testing
    if e%25==0:
        random_model_testing(10,transformer1)

Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 50, 512])
Encoder output shape: torch.Size([64, 50, 512])
Encoder output shape: torch.Size([64, 50, 512])
Encoder output shape: torch.Size([64, 50, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 51, 512])
Encoder output shape: torch.Size([64, 49, 512])
Encoder output shape: torch.Size([64, 49, 512])
Encoder output shape: torch.Size([64, 51

In [ ]:
#more random tests 
random_model_testing(20,transformer1)